In [317]:
import drms
import os
import numpy as np
import math
from tqdm import tqdm

In [294]:
series = 'hmi.sharp_cea_720s'
segments = ['Bt','Br','Bp']
kwlist = ['T_REC','HARPNUM','USFLUX','LON_FWT','LAT_FWT','NOAA_ARS']

c = drms.Client(email = 'test@gmail.com', verbose=True)
out_dir = 'C:\\sharp_data_v2'
if not os.path.exists(out_dir):
    os.mkdir(out_dir)

In [323]:
download_array = []
for sharpnum in tqdm(range(7300,8369)):
    k = c.query('%s[%d]' % (series, sharpnum), key=kwlist)
    try:
        k = k.loc[k.LON_FWT.abs() < 60]
        k = k.loc[k.LAT_FWT.abs() < 60]
        k = k.loc[k.USFLUX > k.USFLUX.quantile(0.75)]
        rec_sample = (k.LON_FWT.pow(2)+k.LAT_FWT.pow(2)).idxmin()
    except AttributeError:
        # Sharp number not present
        continue
    except ValueError:
        # Sharp too close to disk edge
        continue
#     print('{number}{ar}: {flux} @ ({lat},{lon})'.format(
#         number=sharpnum, ar = (f' ({k.NOAA_ARS[rec_sample]})' if k.NOAA_ARS[rec_sample] != 'MISSING' else ''),
#         flux=k.USFLUX[rec_sample], lat=k.LAT_FWT[rec_sample], lon=k.LON_FWT[rec_sample]
#     ))
    k_sample = k.loc[rec_sample]
    t_sample = datetime(k.T_REC[rec_sample])
    fname_mask = '{series}[{sharpnum}][{tstr}]'
    download_array.append(fname_mask.format(
        series=series,sharpnum=sharpnum,tstr=t_sample.queryFormat()
    ) + '{%s}'%(','.join(segments)))


100%|██████████████████████████████████████████████████████████████████████████████| 1069/1069 [25:36<00:00,  1.44s/it]


In [324]:
# JSOC gets mad if you request too many files at once, so we have to split up into smaller requests.
arrs = np.array_split(download_array,math.ceil(len(download_array)/15))

for arr in tqdm(arrs):
    exp_query = ','.join(arr)
#     print('Request: ' + exp_query)
    request = c.export(exp_query)
    request.download(out_dir)

  0%|                                                                                           | 0/46 [00:00<?, ?it/s]

Request: hmi.sharp_cea_720s[7300][2018.08.25_09:00:00]{Bt,Br,Bp},hmi.sharp_cea_720s[7301][2018.09.06_18:00:00]{Bt,Br,Bp},hmi.sharp_cea_720s[7302][2018.09.04_18:24:00]{Bt,Br,Bp},hmi.sharp_cea_720s[7303][2018.09.04_19:48:00]{Bt,Br,Bp},hmi.sharp_cea_720s[7304][2018.09.08_23:48:00]{Bt,Br,Bp},hmi.sharp_cea_720s[7305][2018.09.12_05:48:00]{Bt,Br,Bp},hmi.sharp_cea_720s[7306][2018.09.16_03:00:00]{Bt,Br,Bp},hmi.sharp_cea_720s[7307][2018.09.17_01:00:00]{Bt,Br,Bp},hmi.sharp_cea_720s[7308][2018.09.17_11:12:00]{Bt,Br,Bp},hmi.sharp_cea_720s[7309][2018.09.19_05:36:00]{Bt,Br,Bp},hmi.sharp_cea_720s[7310][2018.09.30_23:12:00]{Bt,Br,Bp},hmi.sharp_cea_720s[7311][2018.10.10_10:24:00]{Bt,Br,Bp},hmi.sharp_cea_720s[7312][2018.10.15_02:36:00]{Bt,Br,Bp},hmi.sharp_cea_720s[7313][2018.10.17_03:36:00]{Bt,Br,Bp},hmi.sharp_cea_720s[7316][2018.10.19_09:48:00]{Bt,Br,Bp}
    record: hmi.sharp_cea_720s[7300][2018.08.25_09:00:00_TAI]{Bp}
  filename: Bp.fits
  -> ..\..\..\..\sharp_data_v2\hmi.sharp_cea_720s.7300.20180825_0

  -> ..\..\..\..\sharp_data_v2\hmi.sharp_cea_720s.7312.20181015_023600_TAI.Bt.fits
    record: hmi.sharp_cea_720s[7312][2018.10.15_02:36:00_TAI]{Br}
  filename: Br.fits
  -> ..\..\..\..\sharp_data_v2\hmi.sharp_cea_720s.7312.20181015_023600_TAI.Br.fits
    record: hmi.sharp_cea_720s[7313][2018.10.17_03:36:00_TAI]{Bp}
  filename: Bp.fits
  -> ..\..\..\..\sharp_data_v2\hmi.sharp_cea_720s.7313.20181017_033600_TAI.Bp.fits
    record: hmi.sharp_cea_720s[7313][2018.10.17_03:36:00_TAI]{Bt}
  filename: Bt.fits
  -> ..\..\..\..\sharp_data_v2\hmi.sharp_cea_720s.7313.20181017_033600_TAI.Bt.fits
    record: hmi.sharp_cea_720s[7313][2018.10.17_03:36:00_TAI]{Br}
  filename: Br.fits
  -> ..\..\..\..\sharp_data_v2\hmi.sharp_cea_720s.7313.20181017_033600_TAI.Br.fits
    record: hmi.sharp_cea_720s[7316][2018.10.19_09:48:00_TAI]{Bp}
  filename: Bp.fits
  -> ..\..\..\..\sharp_data_v2\hmi.sharp_cea_720s.7316.20181019_094800_TAI.Bp.fits
    record: hmi.sharp_cea_720s[7316][2018.10.19_09:48:00_TAI]{Bt}
  file

  2%|█▊                                                                                 | 1/46 [00:33<25:10, 33.58s/it]

  -> ..\..\..\..\sharp_data_v2\hmi.sharp_cea_720s.7316.20181019_094800_TAI.Br.fits
Request: hmi.sharp_cea_720s[7319][2018.10.21_01:36:00]{Bt,Br,Bp},hmi.sharp_cea_720s[7321][2018.11.08_16:36:00]{Bt,Br,Bp},hmi.sharp_cea_720s[7322][2018.11.12_10:36:00]{Bt,Br,Bp},hmi.sharp_cea_720s[7323][2018.11.17_19:00:00]{Bt,Br,Bp},hmi.sharp_cea_720s[7324][2018.11.25_06:12:00]{Bt,Br,Bp},hmi.sharp_cea_720s[7325][2018.12.06_18:36:00]{Bt,Br,Bp},hmi.sharp_cea_720s[7326][2018.12.12_20:48:00]{Bt,Br,Bp},hmi.sharp_cea_720s[7327][2018.12.16_11:36:00]{Bt,Br,Bp},hmi.sharp_cea_720s[7328][2018.12.29_00:24:00]{Bt,Br,Bp},hmi.sharp_cea_720s[7329][2018.12.28_02:00:00]{Bt,Br,Bp},hmi.sharp_cea_720s[7331][2019.01.02_19:36:00]{Bt,Br,Bp},hmi.sharp_cea_720s[7334][2019.01.26_18:48:00]{Bt,Br,Bp},hmi.sharp_cea_720s[7335][2019.01.25_04:24:00]{Bt,Br,Bp},hmi.sharp_cea_720s[7336][2019.01.27_06:36:00]{Bt,Br,Bp},hmi.sharp_cea_720s[7337][2019.02.05_06:12:00]{Bt,Br,Bp}
    record: hmi.sharp_cea_720s[7319][2018.10.21_01:36:00_TAI]{Bp}
  

...

 98%|████████████████████████████████████████████████████████████████████████████████▏ | 45/46 [25:22<00:31, 31.67s/it]

  -> ..\..\..\..\sharp_data_v2\hmi.sharp_cea_720s.8348.20220623_091200_TAI.Br.fits
Request: hmi.sharp_cea_720s[8349][2022.06.24_18:24:00]{Bt,Br,Bp},hmi.sharp_cea_720s[8350][2022.06.28_18:00:00]{Bt,Br,Bp},hmi.sharp_cea_720s[8351][2022.06.26_09:24:00]{Bt,Br,Bp},hmi.sharp_cea_720s[8353][2022.06.23_12:36:00]{Bt,Br,Bp},hmi.sharp_cea_720s[8356][2022.06.25_02:00:00]{Bt,Br,Bp},hmi.sharp_cea_720s[8358][2022.06.25_11:24:00]{Bt,Br,Bp},hmi.sharp_cea_720s[8359][2022.06.24_18:00:00]{Bt,Br,Bp},hmi.sharp_cea_720s[8362][2022.06.28_08:00:00]{Bt,Br,Bp},hmi.sharp_cea_720s[8363][2022.06.26_07:48:00]{Bt,Br,Bp},hmi.sharp_cea_720s[8364][2022.06.28_20:24:00]{Bt,Br,Bp},hmi.sharp_cea_720s[8365][2022.06.29_09:48:00]{Bt,Br,Bp},hmi.sharp_cea_720s[8366][2022.06.27_02:12:00]{Bt,Br,Bp},hmi.sharp_cea_720s[8367][2022.06.27_11:00:00]{Bt,Br,Bp},hmi.sharp_cea_720s[8368][2022.06.28_12:00:00]{Bt,Br,Bp}
    record: hmi.sharp_cea_720s[8349][2022.06.24_18:24:00_TAI]{Bp}
  filename: Bp.fits
  -> ..\..\..\..\sharp_data_v2\hmi.sha

  -> ..\..\..\..\sharp_data_v2\hmi.sharp_cea_720s.8367.20220627_110000_TAI.Bt.fits
    record: hmi.sharp_cea_720s[8367][2022.06.27_11:00:00_TAI]{Br}
  filename: Br.fits
  -> ..\..\..\..\sharp_data_v2\hmi.sharp_cea_720s.8367.20220627_110000_TAI.Br.fits
    record: hmi.sharp_cea_720s[8368][2022.06.28_12:00:00_TAI]{Bp}
  filename: Bp.fits
  -> ..\..\..\..\sharp_data_v2\hmi.sharp_cea_720s.8368.20220628_120000_TAI.Bp.fits
    record: hmi.sharp_cea_720s[8368][2022.06.28_12:00:00_TAI]{Bt}
  filename: Bt.fits
  -> ..\..\..\..\sharp_data_v2\hmi.sharp_cea_720s.8368.20220628_120000_TAI.Bt.fits
    record: hmi.sharp_cea_720s[8368][2022.06.28_12:00:00_TAI]{Br}
  filename: Br.fits


100%|██████████████████████████████████████████████████████████████████████████████████| 46/46 [25:51<00:00, 33.72s/it]

  -> ..\..\..\..\sharp_data_v2\hmi.sharp_cea_720s.8368.20220628_120000_TAI.Br.fits


In [263]:
class datetime:
    def __init__(self, str):
        arr = str.split('_')
        self.date = arr[0].split('.')
        self.time = arr[1].split(':')
        self.TAI = (arr[2] == 'TAI')
        self.year = self.date[0]
        self.month = self.date[1]
        self.day = self.date[2]
        self.hour = self.time[0]
        self.minute = self.time[1]
        self.second = self.time[2]
    def sharpFormat(self):
        string = '{year}{month}{day}_{hour}{minute}{second}'
        if self.TAI:
            string = string + '_TAI'
        return string.format(
            year=self.year,month=self.month,day=self.day,
            hour=self.hour,minute=self.minute,second=self.second
        )
    def queryFormat(self):
        string = '{year}.{month}.{day}_{hour}:{minute}:{second}'
        return string.format(
            year=self.year,month=self.month,day=self.day,
            hour=self.hour,minute=self.minute,second=self.second
        )